In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/application_train.csv


In [2]:
df_bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
df_bureau_bal = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
df_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')

**Engineering a new feature (e.g., previous loan counts) from 'bureau.csv'**
Analyzing the df_bureau, we see that for each loan applicant (i.e., 215354, 162297) we have a list of historical loans of each applicant. This data contains information such as whether the loan is still closed or active, how many days the loan was over due, the amount of credit taken, the credit type, and more.

We can assume that the number of previous loans can potentially impact the probability of default for each applicant. Based the loan ids (i.e., SK_ID_BUREAU) we can perform a number of aggregation operations such as mean, max, min, sum and evalute whether these new features add any value for our model.

In [3]:
df_bureau.head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.00,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.80,42103.80,0.00,0.0,Consumer credit,-22,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.00,0.00,0.0,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.70,0.00,0.00,0.0,Consumer credit,-840,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,4500.00,0.00,0.00,0.0,Credit card,-690,NaN


For applicant 215354 we can see he has 7 active accounts and 5 closed accounts. We can see that x3 loans were credit card, x1 is a car loan, and the rest were consumer credit loans. It is possible that the total number of previous loans held by the applicant may have predictive power on whether the applicant should be approved for his new loan. After all, if an applicant has too many loans he might be in financial distress.

In [4]:
df_bureau[df_bureau['SK_ID_CURR']==215354]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.00,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.8,42103.80,0.00,0.0,Consumer credit,-22,NaN
225157,215354,5714458,Closed,currency 1,-1872,0,-776.0,-1203.0,NaN,0,494550.0,0.00,0.00,0.0,Consumer credit,-696,NaN
225158,215354,5714459,Closed,currency 1,-1734,0,-638.0,-1336.0,NaN,0,1084032.0,0.00,NaN,0.0,Car loan,-1336,NaN
225159,215354,5714460,Closed,currency 1,-1333,0,-1089.0,-987.0,NaN,0,102150.0,NaN,NaN,0.0,Consumer credit,-984,NaN


We create a new dataframe where we group based on the applicants id (SK_ID_CURR) and we apply a count operation to SK_ID_BUREAU. We also choose to use a standard index for our dataframe and rename the column to prev_loan_cnt.

In [5]:
df_prev_loan_cnt = df_bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns={'SK_ID_BUREAU':'prev_loan_cnt'})
df_prev_loan_cnt.head(10)


,SK_ID_CURR,prev_loan_cnt
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3
5,100007,1
6,100008,3
7,100009,18
8,100010,2
9,100011,4


We merge our new dataframe of previous loan acounts (df_prev_loan_cnt) on the applicant id (SK_ID_CURR) with a left join. A left join means that any SK_ID_CURR that is in df_prev_loan_cnt but not in df_train will not be included in the new dataframe, and all rows in df_train will be retained. Where df_train has an applicant, but df_prev_loan_cnt has no information, there will be a NaN. We see this is true for SK_ID_CURR = 100006

In [6]:
df_train = df_train.merge(df_prev_loan_cnt, on='SK_ID_CURR', how='left')
df_train[['SK_ID_CURR','prev_loan_cnt']].head(10)

,SK_ID_CURR,prev_loan_cnt
0,100002,8.0
1,100003,4.0
2,100004,2.0
3,100006,NaN
4,100007,1.0
5,100008,3.0
6,100009,18.0
7,100010,2.0
8,100011,4.0
9,100012,NaN


We fill all entries such as SK_ID_CURR=100006 to 0.

In [8]:
df_train['prev_loan_cnt'] = df_train['prev_loan_cnt'].fillna(0)
df_train[['SK_ID_CURR','prev_loan_cnt']].head(10)

,SK_ID_CURR,prev_loan_cnt
0,100002,8.0
1,100003,4.0
2,100004,2.0
3,100006,0.0
4,100007,1.0
5,100008,3.0
6,100009,18.0
7,100010,2.0
8,100011,4.0
9,100012,0.0


In [9]:
df_train.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,prev_loan_cnt
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,8.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,3.0
6,100009,0,Cash loans,F,Y,Y,1,171000.0,1560726.0,41301.0,...,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0,18.0
7,100010,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
8,100011,0,Cash loans,F,N,Y,0,112500.0,1019610.0,33826.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
9,100012,0,Revolving loans,M,N,Y,0,135000.0,405000.0,20250.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


We use our function plot_kde_hist_var to evaluate the potential explanatory power of the prev_loan_cnt variable. By evaluating the plot and the correlation with Target=True we find that prev_loan_cnt is likely to be a weak predictor of Target.

**Engineering new features from all numeric columns from 'bureau.csv'**
We initially only obtained prev_loan_cnt by grouping together our dataset based on loan application ID (SK_ID_CURR) and using the count operation on the loan IDs (SK_ID_BUREAU).

However, in feature engineering, we don't know which variable might be important and different aggregation calculations such as median, max, min, etc applied to each numeric variable may lead to a better predictor.

We drop out SK_ID_BUREAU because it is only important for prev_loan_cnt and has no meaning even if we applied metrics such as median, min, max, etc sd SK_ID_BUREAU is only useful when applying count.

We perform the same groupby operation as before in 'Previous Loan Counts' on all numeric variables and reset the index.



In [11]:
df_bureau_agg = df_bureau.drop(columns=['SK_ID_BUREAU']).groupby(['SK_ID_CURR'], as_index=False).agg(['count','mean','median','min','max','sum']).reset_index()
df_bureau_agg.head()

SK_ID_CURR DAYS_CREDIT                                        \
                   count         mean  median   min  max   sum   
0     100001           7  -735.000000  -857.0 -1572  -49 -5145   
1     100002           8  -874.000000 -1042.5 -1437 -103 -6992   
2     100003           4 -1400.750000 -1205.5 -2586 -606 -5603   
3     100004           2  -867.000000  -867.0 -1326 -408 -1734   
4     100005           3  -190.666667  -137.0  -373  -62  -572   

  CREDIT_DAY_OVERDUE              ... DAYS_CREDIT_UPDATE                   \
               count mean median  ...             median   min  max   sum   
0                  7  0.0    0.0  ...             -155.0  -155   -6  -652   
1                  8  0.0    0.0  ...             -402.5 -1185   -7 -3999   
2                  4  0.0    0.0  ...             -545.0 -2131  -43 -3264   
3                  2  0.0    0.0  ...             -532.0  -682 -382 -1064   
4                  3  0.0    0.0  ...              -31.0  -121  -11  -163   

  AMT_ANNUITY                                             
        count         mean median  min      max      sum  
0           7  3545.357143    0.0  0.0  10822.5  24817.5  
1           7     0.000000    0.0  0.0      0.0      0.0  
2           0          NaN    NaN  NaN      NaN      0.0  
3           0          NaN    NaN  NaN      NaN      0.0  
4           3  1420.500000    0.0  0.0   4261.5   4261.5  

[5 rows x 73 columns]

We can see that using the groupby and agg functions lead to a Multi-level index. The first level are items such as ['DAYS_CREDIT','CREDIT_DAY_OVERDUE'] whereas the second level are the aggregation functions such as ['count','median','mean','min','max','sum'].

To extract specific columns from the dataframe based on a Multiindex, use the Level 0 name (e.g., DAYS_CREDIT,CREDIT_DAY_OVEDUE) and then the Level 1 name (e.g., mean, median). This returns a Series object. Use the loc function to return a DataFrame object.

In [12]:
df_bureau_agg['DAYS_CREDIT']['count'].head(3)

0    7
1    8
2    4
Name: count, dtype: int64

In [13]:
df_bureau_agg.loc[:,[('DAYS_CREDIT','mean')]].head(3)

,DAYS_CREDIT
,mean
0,-735.00
1,-874.00
2,-1400.75


The xs command is another method for accessing Multi-Level indexes. It allows you to index specifically by listing the column names and the levels that you are referring to in the Multi-index dataframe

In [14]:
df_bureau_agg.xs(('DAYS_CREDIT','mean'),level=(0,1),axis=1).head(3)

,DAYS_CREDIT
,mean
0,-735.00
1,-874.00
2,-1400.75


We can see in the below instance that xs extracted all the mean values for all the Multi-Index entries.

In [15]:
df_bureau_agg.xs(('mean'), level =(1),axis=1).head()

,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,-735.000000,0.0,82.428571,-825.500000,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,-93.142857,3545.357143
1,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,-499.875000,0.000000
2,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.0,254350.125000,0.000000,202500.00000,0.0,-816.000000,NaN
3,-867.000000,0.0,-488.500000,-532.500000,0.000,0.0,94518.900000,0.000000,0.00000,0.0,-532.000000,NaN
4,-190.666667,0.0,439.333333,-123.000000,0.000,0.0,219042.000000,189469.500000,0.00000,0.0,-54.333333,1420.500000


# **Evaluate usefulness of new features**
We merge df_train and df_bureau_agg to evaluate the predictive power of our new features with TARGET. To evaluate the predictive power of the new features we perform the following actions

Merge the new features with the training dataset.

Calculate correlations between the new features and TARGET from the training dataset.

Generate KDE plots of features of populations with Target=0/1.

In [16]:
df_train = df_train.merge(df_bureau_agg, how='left', on='SK_ID_CURR' )
df_train.head()


/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:3878: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,"(DAYS_CREDIT_UPDATE, median)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, max)","(DAYS_CREDIT_UPDATE, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, mean)","(AMT_ANNUITY, median)","(AMT_ANNUITY, min)","(AMT_ANNUITY, max)","(AMT_ANNUITY, sum)"
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,-402.5,-1185.0,-7.0,-3999.0,7.0,0.0,0.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,-545.0,-2131.0,-43.0,-3264.0,0.0,NaN,NaN,NaN,NaN,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,-532.0,-682.0,-382.0,-1064.0,0.0,NaN,NaN,NaN,NaN,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,-783.0,-783.0,-783.0,-783.0,0.0,NaN,NaN,NaN,NaN,0.0


We can see that the merged dataframe contains only one level for the index. Thus the Multi_index from df_bureau_agg has been squashed into a tuple.

In [17]:
df_train.columns

Index([                    'SK_ID_CURR',                         'TARGET',
                   'NAME_CONTRACT_TYPE',                    'CODE_GENDER',
                         'FLAG_OWN_CAR',                'FLAG_OWN_REALTY',
                         'CNT_CHILDREN',               'AMT_INCOME_TOTAL',
                           'AMT_CREDIT',                    'AMT_ANNUITY',
       ...
       ('DAYS_CREDIT_UPDATE', 'median'),    ('DAYS_CREDIT_UPDATE', 'min'),
          ('DAYS_CREDIT_UPDATE', 'max'),    ('DAYS_CREDIT_UPDATE', 'sum'),
               ('AMT_ANNUITY', 'count'),          ('AMT_ANNUITY', 'mean'),
              ('AMT_ANNUITY', 'median'),           ('AMT_ANNUITY', 'min'),
                 ('AMT_ANNUITY', 'max'),           ('AMT_ANNUITY', 'sum')],
      dtype='object', length=195)

In [18]:
colName = list(df_train.columns)
colNewName = []
for col in colName:
  
    if isinstance(col,tuple):
        newCol = '_'.join(col)
        colNewName.append(newCol)
    else:
        colNewName.append(col)  

    
df_train.columns = colNewName
print(df_train.columns)

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'DAYS_CREDIT_UPDATE_median', 'DAYS_CREDIT_UPDATE_min',
       'DAYS_CREDIT_UPDATE_max', 'DAYS_CREDIT_UPDATE_sum', 'AMT_ANNUITY_count',
       'AMT_ANNUITY_mean', 'AMT_ANNUITY_median', 'AMT_ANNUITY_min',
       'AMT_ANNUITY_max', 'AMT_ANNUITY_sum'],
      dtype='object', length=195)


We calculate the correlation of all variables in the dataframe to evaluate whether our new variables from bureau are high magnitude of correlation with TARGET.

In [20]:
corrVal = df_train.corr()['TARGET']
corrValabs = abs(corrVal).sort_values(ascending=False)

In [21]:
print('Highest Correlation:\n{}'.format(corrValabs.head(15)))

Highest Correlation:
TARGET                         1.000000
EXT_SOURCE_3                   0.178919
EXT_SOURCE_2                   0.160472
EXT_SOURCE_1                   0.155317
DAYS_CREDIT_mean               0.089729
DAYS_CREDIT_median             0.085414
DAYS_BIRTH                     0.078239
DAYS_CREDIT_min                0.075248
DAYS_CREDIT_UPDATE_mean        0.068927
DAYS_CREDIT_UPDATE_median      0.067338
REGION_RATING_CLIENT_W_CITY    0.060893
REGION_RATING_CLIENT           0.058899
DAYS_ENDDATE_FACT_min          0.055887
DAYS_LAST_PHONE_CHANGE         0.055218
DAYS_CREDIT_ENDDATE_sum        0.053735
Name: TARGET, dtype: float64


In [22]:
print('Lowest Correlation:\n{}'.format(corrValabs.tail(15)))

Lowest Correlation:
FLAG_DOCUMENT_10              0.001414
AMT_ANNUITY_mean              0.001391
FLAG_DOCUMENT_19              0.001358
AMT_ANNUITY_max               0.001120
AMT_REQ_CREDIT_BUREAU_HOUR    0.000930
AMT_REQ_CREDIT_BUREAU_WEEK    0.000788
FLAG_DOCUMENT_12              0.000756
AMT_CREDIT_SUM_DEBT_mean      0.000637
FLAG_MOBIL                    0.000534
FLAG_CONT_MOBILE              0.000370
FLAG_DOCUMENT_5               0.000316
AMT_CREDIT_SUM_DEBT_min       0.000242
FLAG_DOCUMENT_20              0.000215
CNT_CREDIT_PROLONG_min        0.000182
AMT_CREDIT_SUM_OVERDUE_min    0.000003
Name: TARGET, dtype: float64


We see that a few of our new variables have higher correlations with TARGET such as (DAYS_CREDIT, x), (DAYS_CREDIT_UPDATE,x), thus we can run our KDE plot

We plot the distributions for these new feature variables that exhibit higher correlations with TARGET. Bear in mind that the correlations are still around 0.06 - 0.09 which are still quite low in the scheme of things.

The shapes of the KDEs of these different features for the TARGET=0 and TARGET=1 populations are quite different but this may be due to the imbalanced population of the dataset

In [25]:


newFeat = ['DAYS_CREDIT_mean','DAYS_CREDIT_median','DAYS_CREDIT_min','DAYS_CREDIT_UPDATE_mean']


Functions for aggregating numeric columns
We write a function that aggregates all the numeric columns in a dataframe, and collapses the multi-index dataframe into a single index.

It is often easier to work with a Single Index rather than a multi index dataframe (i.e., When merging a multi-index with a single-index dataframe, multi-index values are converted into tuples).



In [27]:
df_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


# Using sklearn's OneHotEncoder
We apply scikit-learn's OneHotEncoder which as of version 20 can now function on string variables.

We can see from previous cells that CREDIT_ACTIVE consists of x4 unique variables. Thus, we expect that if we apply OneHotEncoder, this would produce a matrix with x4 columns.

Tip

OneHotEncoder only accepts DataFrames, not a Series so use [[ ]] when selecting a single column(s).
Use the handle_unknown='ignore' functionality of OneHotEncoder to ignore issues such as:
Unknown categories/missing values in the test set that are not in the training set.
OneHotEncoder can perform transform and ignore unknown categories/missing values on the test data.
OneHotEncoder cannot perform fit_transform on unknown categories/missing values for the training data.

In [30]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

credit_act = df_bureau[['CREDIT_ACTIVE']]
credit_act_OHE = ohe.fit_transform(credit_act)

In [31]:
print(credit_act_OHE)

[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [32]:
featnames = ohe.get_feature_names()
print(featnames)


['x0_Active' 'x0_Bad debt' 'x0_Closed' 'x0_Sold']


In [33]:
df_credit_ohe = pd.DataFrame(credit_act_OHE,columns=featnames)
df_credit_ohe.head()

,x0_Active,x0_Bad debt,x0_Closed,x0_Sold
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0


In [34]:
df_bureau['CREDIT_ACTIVE'].head()

0    Closed
1    Active
2    Active
3    Active
4    Active
Name: CREDIT_ACTIVE, dtype: object

We can see that the OneHotEncoder performs an accurate operation by comparing the results between df_credit_ohe and df_bureau['CREDIT_ACTIVE'].

We also verify this by performing the inverse_transform applied to the transformed OHE array back to its original labeled values.

In [35]:
ohe.inverse_transform(credit_act_OHE)[:5]

array([['Closed'],
       ['Active'],
       ['Active'],
       ['Active'],
       ['Active']], dtype=object)

**OneHotEncoder on a full DataFrame**

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [36]:
df_bureau_categorical = ohe.fit_transform(df_bureau.select_dtypes('object'))
print(df_bureau_categorical.shape)
df_bureau_categorical[:5]

(1716428, 23)


array([[0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]])

In [37]:
df_bureau_categorical_origin = ohe.inverse_transform(df_bureau_categorical)
print(df_bureau_categorical_origin)

[['Closed' 'currency 1' 'Consumer credit']
 ['Active' 'currency 1' 'Credit card']
 ['Active' 'currency 1' 'Consumer credit']
 ...
 ['Closed' 'currency 1' 'Consumer credit']
 ['Closed' 'currency 1' 'Consumer credit']
 ['Closed' 'currency 1' 'Microloan']]


In [39]:
featnames = ohe.get_feature_names()
print(featnames)


['x0_Active' 'x0_Bad debt' 'x0_Closed' 'x0_Sold' 'x1_currency 1'
 'x1_currency 2' 'x1_currency 3' 'x1_currency 4' 'x2_Another type of loan'
 'x2_Car loan' 'x2_Cash loan (non-earmarked)' 'x2_Consumer credit'
 'x2_Credit card' 'x2_Interbank credit' 'x2_Loan for business development'
 'x2_Loan for purchase of shares (margin lending)'
 'x2_Loan for the purchase of equipment'
 'x2_Loan for working capital replenishment' 'x2_Microloan'
 'x2_Mobile operator loan' 'x2_Mortgage' 'x2_Real estate loan'
 'x2_Unknown type of loan']


In [40]:
df_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


# Using panda's get_dummies
We use the get_dummies from pandas to perform one hot encoding.

In [41]:
df_categorical = pd.get_dummies(df_bureau.select_dtypes('object'))

df_categorical['SK_ID_CURR'] = df_bureau['SK_ID_CURR']  
df_categorical.head()

,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,SK_ID_CURR
0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,215354


In [42]:
df_categorical_grouped = df_categorical.groupby('SK_ID_CURR').agg(['sum','mean'])
df_categorical_grouped.head()

CREDIT_ACTIVE_Active           CREDIT_ACTIVE_Bad debt       \
                            sum      mean                    sum mean   
SK_ID_CURR                                                              
100001                        3  0.428571                      0  0.0   
100002                        2  0.250000                      0  0.0   
100003                        1  0.250000                      0  0.0   
100004                        0  0.000000                      0  0.0   
100005                        2  0.666667                      0  0.0   

           CREDIT_ACTIVE_Closed           CREDIT_ACTIVE_Sold       \
                            sum      mean                sum mean   
SK_ID_CURR                                                          
100001                        4  0.571429                  0  0.0   
100002                        6  0.750000                  0  0.0   
100003                        3  0.750000                  0  0.0   
100004                        2  1.000000                  0  0.0   
100005                        1  0.333333                  0  0.0   

           CREDIT_CURRENCY_currency 1       ... CREDIT_TYPE_Microloan       \
                                  sum mean  ...                   sum mean   
SK_ID_CURR                                  ...                              
100001                              7  1.0  ...                     0  0.0   
100002                              8  1.0  ...                     0  0.0   
100003                              4  1.0  ...                     0  0.0   
100004                              2  1.0  ...                     0  0.0   
100005                              3  1.0  ...                     0  0.0   

           CREDIT_TYPE_Mobile operator loan      CREDIT_TYPE_Mortgage       \
                                        sum mean                  sum mean   
SK_ID_CURR                                                                   
100001                                    0  0.0                    0  0.0   
100002                                    0  0.0                    0  0.0   
100003                                    0  0.0                    0  0.0   
100004                                    0  0.0                    0  0.0   
100005                                    0  0.0                    0  0.0   

           CREDIT_TYPE_Real estate loan      CREDIT_TYPE_Unknown type of loan  \
                                    sum mean                              sum   
SK_ID_CURR                                                                      
100001                                0  0.0                                0   
100002                                0  0.0                                0   
100003                                0  0.0                                0   
100004                                0  0.0                                0   
100005                                0  0.0                                0   

                 
           mean  
SK_ID_CURR       
100001      0.0  
100002      0.0  
100003      0.0  
100004      0.0  
100005      0.0  

[5 rows x 46 columns]

In [43]:
df_bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [44]:
df_bureau_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int64 
 1   MONTHS_BALANCE  int64 
 2   STATUS          object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [45]:
df_bureau_bal['STATUS'].unique()

array(['C', '0', 'X', '1', '2', '3', '5', '4'], dtype=object)

In [46]:
df = df_bureau_bal
groupby_var = 'SK_ID_BUREAU'
df_name = 'bureau_bal'
drop_var_list = list()